In [26]:
import langdetect
from gtts import gTTS
from gtts.lang import tts_langs
from polyglot.detect import Detector
from io import BytesIO
import torchaudio
import torch
import pycld2 as cld
from IPython.display import Audio
import icu
import unicodedata

In [15]:
langs = ["en", "th", "ko", "zh", "ja"]
charsets = [
    set(icu.LocaleData(lang).getExemplarSet(icu.USET_ADD_CASE_MAPPINGS, icu.ULocaleDataExemplarSetType.ES_STANDARD))
    for lang in langs
]

In [22]:
def split_lang(s: str) -> list[tuple[str, str]]:
    res: list[tuple[str, str]] = []
    last_lang = "unk"
    stk = ""
    for c in s:
        cur_lang = "unk"
        for lang, charset in zip(langs, charsets):
            if c in charset:
                cur_lang = lang
                break
        print(c, cur_lang)
        if cur_lang == "unk" or cur_lang == last_lang:
            stk += c
        else:
            res.append((last_lang if last_lang != "unk" else "en", stk.strip()))
            last_lang = cur_lang
            stk = c
    res.append((last_lang if last_lang != "unk" else "en", stk.strip()))
    return res

In [23]:
s = "NewJeans (뉴진스) 'Super Shy' Official MV"
# s = "hello python curry มิลวย"
s = "플라스틱 강남역 4번 출구_New MV | k-pop | 인디밴드 | 감성음악 |  어쿠스틱 | 카페음악 | Acoustic"

In [29]:
s = "플라스틱 강남역 4번 출구_New MV | k-pop | 인디밴드 | 감성음악 |  어쿠스틱 | 카페음악 | Acoustic"
for c in unicodedata.normalize("NFC", s):
    print(c)

ᄑ
ᅳ
ᆯ
ᄅ
ᅡ
ᄉ
ᅳ
ᄐ
ᅵ
ᆨ
 
ᄀ
ᅡ
ᆼ
ᄂ
ᅡ
ᆷ
ᄋ
ᅧ
ᆨ
 
4
ᄇ
ᅥ
ᆫ
 
ᄎ
ᅮ
ᆯ
ᄀ
ᅮ
_
N
e
w
 
M
V
 
|
 
k
-
p
o
p
 
|
 
ᄋ
ᅵ
ᆫ
ᄃ
ᅵ
ᄇ
ᅢ
ᆫ
ᄃ
ᅳ
 
|
 
ᄀ
ᅡ
ᆷ
ᄉ
ᅥ
ᆼ
ᄋ
ᅳ
ᆷ
ᄋ
ᅡ
ᆨ
 
|
 
 
ᄋ
ᅥ
ᄏ
ᅮ
ᄉ
ᅳ
ᄐ
ᅵ
ᆨ
 
|
 
ᄏ
ᅡ
ᄑ
ᅦ
ᄋ
ᅳ
ᆷ
ᄋ
ᅡ
ᆨ
 
|
 
A
c
o
u
s
t
i
c


In [30]:
split_lang(s)

ᄑ unk
ᅳ unk
ᆯ unk
ᄅ unk
ᅡ unk
ᄉ unk
ᅳ unk
ᄐ unk
ᅵ unk
ᆨ unk
  unk
ᄀ unk
ᅡ unk
ᆼ unk
ᄂ unk
ᅡ unk
ᆷ unk
ᄋ unk
ᅧ unk
ᆨ unk
  unk
4 unk
ᄇ unk
ᅥ unk
ᆫ unk
  unk
ᄎ unk
ᅮ unk
ᆯ unk
구 ko
_ unk
N en
e en
w en
  unk
M en
V en
  unk
| unk
  unk
k en
- unk
p en
o en
p en
  unk
| unk
  unk
인 ko
디 ko
밴 ko
드 ko
  unk
| unk
  unk
감 ko
성 ko
음 ko
악 ko
  unk
| unk
  unk
  unk
어 ko
쿠 ko
스 ko
틱 ko
  unk
| unk
  unk
카 ko
페 ko
음 ko
악 ko
  unk
| unk
  unk
A en
c en
o en
u en
s en
t en
i en
c en


[('en', '플라스틱 강남역 4번 출'),
 ('ko', '구_'),
 ('en', 'New MV | k-pop |'),
 ('ko', '인디밴드 | 감성음악 |  어쿠스틱 | 카페음악 |'),
 ('en', 'Acoustic')]

In [6]:
ttss = []
for lang, word in [("th", s)]: # split_lang(s):
    audio_fd = BytesIO()
    gTTS(word, lang=lang).write_to_fp(audio_fd)
    audio_fd.seek(0)
    audio_tts, tts_rate = torchaudio.load(audio_fd, format="mp3")  # type: ignore
    audio_fd.close()
    audio_tts = torchaudio.functional.resample(audio_tts, tts_rate, 44100)
    ttss.append(audio_tts)

In [7]:
x = torch.cat(ttss, dim=-1)

In [8]:
Audio(x.numpy(), rate=44100)